In [42]:
import requests,json,re
from requests_toolbelt import MultipartEncoder

In [43]:
session = requests.Session()
user_agent = r'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'

In [322]:
def get_problems():
    url = "https://leetcode.com/api/problems/all/"

    headers = {'User-Agent': user_agent, 'Connection': 'keep-alive'}
    resp = session.get(url, headers = headers, timeout = 10)
       
    question_list = json.loads(resp.content.decode('utf-8'))

    allTitle=[]
    for question in question_list['stat_status_pairs']:
        question_id = question['stat']['question_id']
        question_slug = question['stat']['question__title_slug']
        question_status = question['status']
        level = question['difficulty']['level']
        if question['paid_only']:
            continue
        allTitle.append(question_slug)
    return allTitle   

def get_problem_by_slug(slug):
    url = "https://leetcode.com/graphql"
    params = {'operationName': "getQuestionDetail",
        'variables': {'titleSlug': slug},
        'query': '''query getQuestionDetail($titleSlug: String!) {
            question(titleSlug: $titleSlug) {
                questionId
                questionFrontendId
                questionTitle
                questionTitleSlug
                content
                difficulty
                stats
                similarQuestions
                categoryTitle
                topicTags {
                        name
                        slug
                }
            }
        }'''
    }

    json_data = json.dumps(params).encode('utf8')
                        
    headers = {'User-Agent': user_agent, 'Connection': 
        'keep-alive', 'Content-Type': 'application/json',
        'Referer': 'https://leetcode.com/problems/' + slug}
    resp = session.post(url, data = json_data, headers = headers, timeout = 10)
    content = resp.json()

    # 题目详细信息
    # question = content['data']['question']['questionTitleSlug']
    # content=content['data']['question']['content']
    return content['data']['question']['content']
    

In [309]:
def get_sol_by_slug(slug,start):
    url = f"https://leetcode.com/graphql/{slug}/"
    params = {'operationName': "communitySolutions",
        'variables': {'query': "", 'languageTags': ["cpp"], 'topicTags': [], 'questionSlug': slug, 'skip': start*15,'first':15,'orderBy':'hot'},
        'query': '''query communitySolutions($questionSlug: String!, $skip: Int!, $first: Int!, $query: String, $orderBy: TopicSortingOption, $languageTags: [String!], $topicTags: [String!]) {
            questionSolutions(filters: {questionSlug: $questionSlug, skip: $skip, first: $first, query: $query, orderBy: $orderBy, languageTags: $languageTags, topicTags: $topicTags}) {
                hasDirectResults
                totalNum
                solutions {
                    id
                    title
                    commentCount
                    topLevelCommentCount
                    viewCount
                    pinned
                    isFavorite
                    solutionTags {
                        name
                        slug
                    }    
                }
            }
        }    
        '''
    }

    json_data = json.dumps(params).encode('utf8')
                        
    headers = {'User-Agent': user_agent, 'Connection': 
        'keep-alive', 'Content-Type': 'application/json',
        'Referer': 'https://leetcode.com/problems/' + slug+'/solutions'}
    resp = session.post(url, data = json_data, headers = headers, timeout = 10)
    content = resp.json()
    return content['data']['questionSolutions']['solutions']

In [310]:
def getSolution(slug,id,t):
    url = f"https://leetcode.com/graphql/{slug}/solution"
    params = {'operationName': "communitySolution",
        'variables': {'topicId': id},
        'query': '''query communitySolution($topicId: Int!) {
            topic(id: $topicId) {
                post {
                    id
                    voteCount
                    voteStatus
                    content
                }
            }
        }   
        '''
    }

    json_data = json.dumps(params).encode('utf8')
                        
    headers = {'User-Agent': user_agent, 'Connection': 
        'keep-alive', 'Content-Type': 'application/json',
        'Referer': 'https://leetcode.com/problems/' + slug+'/solutions/'+str(id)+'/'+t}
    resp = session.post(url, data = json_data, headers = headers, timeout = 10)
    content = resp.json()


    return content['data']['topic']['post']['content']

In [324]:
import csv
def writecsv(htmlItems,t,title,id,counter,leetcodeDesc):
    # with open(f"test\\{title}_{t}_{id}_{counter}.txt",'w')as f:
    #     for i in htmlItems:
    #         i=i.replace("\\t","")
    #         f.write(i+'\n')
    with open('output.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([t, leetcodeDesc, htmlItems])

In [328]:
def RemoveTag(htmlString):
    pattern = re.compile('<.*?>')
    result = re.sub(pattern, '', htmlString)
    return result

In [329]:
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['question', 'context', 'answer'])
titles=get_problems()
for t in titles:
    leetcodeDesc=RemoveTag(get_problem_by_slug(t))#取t的題目敘述，t是題目標題
    print(leetcodeDesc)
    for i in range(1):
        try:
            temp=get_sol_by_slug(t,i)
            for j in iter(temp):
                title=j['title']
                title=re.sub('[^\w]'," ",title).lower()
                title=re.sub('\s+',"-",title)
                if(title[0]=='-'):
                    title=title[1:]
                if(title[-1]=='-'):
                    title=title[:-1]
                print(f"{t}, {j['id']}")
                htmlItems=getSolution(t,j['id'],title)
                counter=1
                while(True):
                    index=htmlItems.find("```")
                    if(index!=-1):
                        index=htmlItems.find("class")
                        if index==-1:
                            index=htmlItems.find("```")
                            htmlItems=htmlItems[index+3:]
                            continue
                        htmlItems=htmlItems[index:]
                        index=htmlItems.find("```")
                        temp=htmlItems[:index]
                        temp=temp.replace("\\n"," ")
                        htmlItems=htmlItems[index+3:]
                        writecsv(temp,t,title,j['id'],counter,leetcodeDesc)
                        counter+=1
                    else:
                        break
        except:
            continue
            

You are given a 0-indexed array maxHeights of n integers.

You are tasked with building n towers in the coordinate line. The ith tower is built at coordinate i and has a height of heights[i].

A configuration of towers is beautiful if the following conditions hold:


	1 &lt;= heights[i] &lt;= maxHeights[i]
	heights is a mountain array.


Array heights is a mountain if there exists an index i such that:


	For all 0 &lt; j &lt;= i, heights[j - 1] &lt;= heights[j]
	For all i &lt;= k &lt; n - 1, heights[k + 1] &lt;= heights[k]


Return the maximum possible sum of heights of a beautiful configuration of towers.

&nbsp;
Example 1:


Input: maxHeights = [5,3,4,1,1]
Output: 13
Explanation: One beautiful configuration with a maximum sum is heights = [5,3,3,1,1]. This configuration is beautiful since:
- 1 &lt;= heights[i] &lt;= maxHeights[i]  
- heights is a mountain of peak i = 0.
It can be shown that there exists no other beautiful configuration with a sum of heights greater than 13.

Example

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
df=pd.read_csv("output.csv")
df.head()
df_train,df_test=train_test_split(df,test_size=0.3)


df_train.to_json("mydata_train.jsonl", orient='records', lines=True)
df_test.to_json("mydata_eval.jsonl", orient='records', lines=True)

In [5]:
print(df_train)

                                                question  \
2199               minimize-the-total-price-of-the-trips   
11511                                       car-fleet-ii   
17973                number-of-visible-people-in-a-queue   
30794                              contains-duplicate-ii   
1820                find-the-losers-of-the-circular-game   
...                                                  ...   
15744    number-of-times-binary-string-is-prefix-aligned   
12537                                      stone-game-vi   
30833                                 contains-duplicate   
34319                                   integer-to-roman   
17704  longest-arithmetic-subsequence-of-given-differ...   

                                                 context  \
2199   <p>There exists an undirected and unrooted tre...   
11511  <p>There are <code>n</code> cars traveling at ...   
17973  <p>There are <code>n</code> people standing in...   
30794  <p>Given an integer array <code>